# Content

- Crawl dữ liệu thô từ web, vào từng file .txt

- Đọc và xử lý dữ liệu thành bộ dữ liệu huấn luyện

- Sử dụng VnCoreNLP để gắn nhãn cho dữ liệu

- Dữ liệu được lưu dưới dạng CoNLL (Token, label)

- Từ file dữ liệu dã tạo, huấn luyện bằng mô hình spaCy (NER)

In [27]:
!pip install vncorenlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 30.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645932 sha256=5db0237ed40d5ce23c623a1fba965a2723d63689b4f2da8f9b0a4faf900e744b
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561fd4aab55dc0480b9d7a131c6aaa573a3fa
Successfully built vncorenlp


In [28]:
!git clone https://github.com/vncorenlp/VnCoreNLP.git


Cloning into 'VnCoreNLP'...
remote: Enumerating objects: 259, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 259 (delta 17), reused 33 (delta 11), pack-reused 212 (from 1)
Receiving objects: 100% (259/259), 237.79 MiB | 19.00 MiB/s, done.
Resolving deltas: 100% (93/93), done.
Updating files: 100% (34/34), done.


In [29]:
!pip install pyvi

## Load VnCoreNLP Model

In [30]:
from vncorenlp import VnCoreNLP
import os

# Use VnCoreNLP to make label for raw data
vncorenlp = VnCoreNLP("VnCoreNLP/VnCoreNLP-1.1.1.jar")

## Read data from text file

In [31]:
# Data is saved in many file txt
def read_txt_files_in_folder(folder_path):
    file_contents = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                file_contents.append(file.read())
    return file_contents

folder_path = "vnexpress"
texts = read_txt_files_in_folder(folder_path)

# use (NER) function in VnCoreNLP
def extract_entities_from_text(text):
    try:
        entity_result = vncorenlp.ner(text)
        return entity_result
    except KeyError as e:
        print(f"KeyError: {e}")
        return None

# use extract_entities_from_text function for data
annotated_texts = [extract_entities_from_text(text) for text in texts]

# format to CoNLL type
def format_to_conll(annotated_text):
    conll_data = []
    for sentence in annotated_text:
        for word, label in sentence:
            token = word.replace("_", " ")  # Gộp các từ tách bởi dấu _
            conll_data.append(f"{token}\t{label}")
        conll_data.append("")
    return "\n".join(conll_data)

conll_datasets = [format_to_conll(annotated) for annotated in annotated_texts if annotated is not None]

# Save dataset
with open('training_data.conll', 'w', encoding='utf-8') as file:
    file.write("\n\n".join(conll_datasets))

print("Dữ liệu đã được lưu vào training_data.conll")

Dữ liệu đã được lưu vào training_data.conll


### Count label in data

In [32]:
label_count = {"O": 0, "B-PER": 0, "I-PER": 0, "B-LOC": 0, "I-LOC": 0, "B-MISC": 0, "I-MISC": 0, "B-ORG": 0, "I-ORG": 0}

for annotated in annotated_texts:
    if annotated is None:  # Bỏ qua nếu giá trị là None
        continue
    for sentence in annotated:
        for _, label in sentence:
            label_count[label] += 1

print("Label distribution:", label_count)


Label distribution: {'O': 267847, 'B-PER': 3348, 'I-PER': 170, 'B-LOC': 7196, 'I-LOC': 3841, 'B-MISC': 138, 'I-MISC': 141, 'B-ORG': 2281, 'I-ORG': 3982}


Old dataset (This data set is noisy, training the model does not give good results)

In [ ]:
# label_count = {"O": 0, "B-PER": 0, "I-PER": 0, "B-LOC": 0, "I-LOC": 0, "B-MISC": 0, "I-MISC": 0, "B-ORG": 0, "I-ORG": 0}

# for annotated in annotated_texts:
#     if annotated is None:  # Bỏ qua nếu giá trị là None
#         continue
#     for sentence in annotated:
#         for _, label in sentence:
#             label_count[label] += 1

# print("Label distribution:", label_count)


Label distribution: {'O': 129945, 'B-PER': 658, 'I-PER': 30, 'B-LOC': 989, 'I-LOC': 280, 'B-MISC': 171, 'I-MISC': 174, 'B-ORG': 229, 'I-ORG': 330}


## Train with spacy (Run from here if having dataset)

In [1]:
import spacy
from spacy.training import Example

def conll_to_spacy_format(conll_file):
    texts = []
    annotations = []

    with open(conll_file, 'r', encoding='utf-8') as file:
        text = []
        entities = []
        start_idx = None
        for line in file:
            line = line.strip()
            if not line:  # space line
                if text:
                    # annotations for current sentence
                    annotations.append({"entities": entities})
                    texts.append(" ".join(text))
                text = []
                entities = []
                continue
            word, label = line.split('\t')
            text.append(word)
            if label != 'O':  # Remove O label
                start_idx = len(" ".join(text)) - len(word)
                entities.append((start_idx, start_idx + len(word), label))

    return list(zip(texts, annotations))


# data for train with correct format for spacy
train_data = conll_to_spacy_format("training_data.conll")


In [2]:
# In 5 dòng đầu tiên của train_data
for i, (text, annotation) in enumerate(train_data[:5]):  # Chỉ lấy 5 phần tử đầu tiên
    print(f"Example {i + 1}:")
    print(f"Text: {text}")
    print(f"Annotations: {annotation}")
    print("-" * 50)


Example 1:
Text: Đường sắt Bắc Nam chia cắt vì sạt lở Mưa lớn khiến tuyến đường sắt Bắc Nam qua tỉnh Quảng Trị bị trôi đá , xói nền , nhiều đoàn tàu phải dừng tại ga Đông Hà và Đồng Hới để chờ gia cố đường.Theo Tổng công ty Đường sắt Việt Nam , bão Trà Mi gây mưa lớn , làm ngập tuyến đường sắt qua Quảng Trị .
Annotations: {'entities': [(10, 17, 'B-PER'), (79, 83, 'B-LOC'), (84, 93, 'I-LOC'), (146, 148, 'B-LOC'), (149, 156, 'I-LOC'), (160, 168, 'B-LOC'), (194, 206, 'B-ORG'), (207, 216, 'I-ORG'), (217, 225, 'I-ORG'), (282, 291, 'B-LOC')]}
--------------------------------------------------
Example 2:
Text: Đến sáng nay , nước rút làm trôi đá , xói nền , dịch chuyển ray nhiều đoạn ở huyện Vĩnh Linh.Đoạn đường sắt bị xói lở , trơ ray .
Annotations: {'entities': [(77, 82, 'B-LOC'), (83, 87, 'I-LOC')]}
--------------------------------------------------
Example 3:
Text: Ảnh : VNRNgành đường sắt đã phong toả đoạn Sa Lung - Tiên An , chuyển tải hành khách giữa ga Đông Hà ( Quảng Trị ) và ga Đồng

In [5]:
# !pip install pyvi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.0 MB/s eta 0:00:00


In [4]:
import spacy
from spacy.training import Example

# model
nlp = spacy.blank("vi") #need install pyvi

# pipeline is NER for this model
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe("ner", last=True)

# add label NER
for _, annotations in train_data:
    for ent in annotations["entities"]:
        # Thêm nhãn vào pipeline NER
        nlp.get_pipe("ner").add_label(ent[2])

# check the pipe line
print(nlp.pipe_names)


['ner']


In [8]:
from spacy.training import offsets_to_biluo_tags

# Check the match between entities and text
for text, annotations in train_data:
    doc = nlp.make_doc(text)  # Tạo Doc từ văn bản
    entities = annotations["entities"]


    # biluo_tags = offsets_to_biluo_tags(doc, entities)
    # print(f"Text: {text}")
    # print(f"Entities: {entities}")
    # print(f"BILUO Tags: {biluo_tags}")


### Data processing


In [6]:
from spacy.training import Example
from sklearn.model_selection import train_test_split
from spacy.training import Example, offsets_to_biluo_tags
train_data, test_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Convert training data to Example objects
train_examples = []
for text, annotations in train_data:
    # Create Doc from text
    doc = nlp.make_doc(text)

    # Check alignment of entities using offsets_to_biluo_tags
    try:
        # This will raise an error if the entity positions don't align with the text
        biluo_tags = offsets_to_biluo_tags(doc, annotations['entities'])

        # If no error, create Example object
        example = Example.from_dict(doc, annotations)
        train_examples.append(example)
    except ValueError as e:
        # Handle misalignment error (optional: log or inspect the problematic data)
        print(f"Misalignment error for text: {text}")
        print(f"Error: {e}")

# Optional: Check first example
print(train_examples[0].text)


Nguyên nhân do nhà đầu tư không đủ năng lực.Hướng tuyến đường nối .


In [9]:
# In 5 Example đầu tiên trong train_examples
for i, example in enumerate(train_examples[:5]):
    print(f"Example {i + 1}:")
    print(f"Text: {example.reference.text}") #RAW TEXT

    # SHOW ENTITITES
    entities = [
        (ent.text, ent.start_char, ent.end_char, ent.label_) 
        for ent in example.reference.ents
    ]
    print(f"Entities: {entities}")
    print("-" * 50)


Example 1:
Text: Nguyên nhân do nhà đầu tư không đủ năng lực.Hướng tuyến đường nối .
Entities: []
--------------------------------------------------
Example 2:
Text: Khu vực này cách sân bay Phù Cát ( nơi máy bay xuất phát ) ở Bình Định hơn 100 km .
Entities: [('sân bay', 17, 24, 'B-LOC'), ('Phù Cát', 25, 32, 'I-LOC'), ('Bình Định', 61, 70, 'B-LOC')]
--------------------------------------------------
Example 3:
Text: Để đạt mục tiêu nêu trên , Tổng Bí thư yêu cầu đột phá hơn nữa về thể chế , tháo gỡ hết các khó khăn , vướng mắc , điểm nghẽn để khơi thông mọi nguồn lực và cải cách hành chính mạnh mẽ , kiến tạo môi trường thuận lợi để phát triển.Ghi nhận những kết quả bước đầu về đổi mới tư duy sâu sắc trong xây dựng pháp luật , nhưng Tổng Bí thư cho rằng phía trước còn rất nhiều việc phải làm để khơi thông " điểm nghẽn của điểm nghẽn này " .
Entities: [('Tổng Bí thư', 322, 333, 'B-PER')]
--------------------------------------------------
Example 4:
Text: " Sau khi sắp xếp thu gọn đầu mố

## Train model

In [ ]:
import random

# Begin training
optimizer = nlp.begin_training()

# Train the model for a few epochs
for epoch in range(10):  # Adjust the number of epochs as needed
    print(f"Epoch {epoch+1}")
    random.shuffle(train_examples)  # Shuffle data for better training performance
    losses = {}

    for example in train_examples:
        nlp.update([example], losses=losses)  # Update model weights
    print(f"Losses: {losses}")


Epoch 1
Losses: {'ner': 10160.02453914525}
Epoch 2
Losses: {'ner': 6271.259646323885}
Epoch 3
Losses: {'ner': 5229.01031204718}
Epoch 4
Losses: {'ner': 4416.784663036895}
Epoch 5
Losses: {'ner': 3700.6436001176676}
Epoch 6
Losses: {'ner': 3292.6315551880743}
Epoch 7
Losses: {'ner': 3011.8336950316843}
Epoch 8
Losses: {'ner': 2732.501388711985}
Epoch 9
Losses: {'ner': 2559.772333797187}
Epoch 10
Losses: {'ner': 2410.552204241079}


In [ ]:
# Train from exist model
import random
import spacy
nlp = spacy.load("IE Model")
# Begin training
# optimizer = nlp.begin_training()

# Train the model for a few epochs
for epoch in range(10):  # Adjust the number of epochs as needed
    print(f"Epoch {epoch+1}")
    random.shuffle(train_examples)  # Shuffle data for better training performance
    losses = {}

    for example in train_examples:
        nlp.update([example], losses=losses)  # Update model weights
    print(f"Losses: {losses}")


### Evaluate model


In [ ]:
from spacy.training import Example, offsets_to_biluo_tags

# Convert test data to Example objects
test_examples = []
for text, annotations in test_data:
    doc = nlp.make_doc(text)  # Create Doc from text

    # Check alignment of entities using offsets_to_biluo_tags
    try:
        # This will raise an error if the entity positions don't align with the text
        biluo_tags = offsets_to_biluo_tags(doc, annotations['entities'])

        # If no error, create Example object
        example = Example.from_dict(doc, annotations)
        test_examples.append(example)
    except ValueError as e:
        # Handle misalignment error (optional: log or inspect the problematic data)
        print(f"Misalignment error for text: {text}")
        print(f"Error: {e}")

# Evaluate the model
accuracy = 0
for example in test_examples:
    # Get predictions for the test example
    predicted_doc = nlp(example.text)
    correct = 0
    total = 0

    # Compare the predicted and actual entities
    for ent in example.reference.ents:
        total += 1
        if ent.text in [e.text for e in predicted_doc.ents]:
            correct += 1

    if total > 0:
        accuracy += correct / total

# Calculate overall accuracy
accuracy /= len(test_examples)
print(f"Accuracy: {accuracy:.4f}")


In [25]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the model
true_entities = []
pred_entities = []

for example in test_examples:
    # Get predictions for the test example
    predicted_doc = nlp(example.text)

    # Collect true entities and their labels
    true_entities += [(ent.text, ent.label_) for ent in example.reference.ents]

    # Collect predicted entities and their labels
    pred_entities += [(ent.text, ent.label_) for ent in predicted_doc.ents]

# Convert to sets for exact match evaluation
true_set = set(true_entities)
pred_set = set(pred_entities)

# Precision, Recall, F1 based on set overlap
tp = len(true_set & pred_set)  # True Positives
fp = len(pred_set - true_set)  # False Positives
fn = len(true_set - pred_set)  # False Negatives

precision = tp / (tp + fp) if tp + fp > 0 else 0.0
recall = tp / (tp + fn) if tp + fn > 0 else 0.0
f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0.0

# Print results
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")


Precision: 0.8588, Recall: 0.9018, F1 Score: 0.8798


In [26]:
from sklearn.metrics import classification_report

y_true = []
y_pred = []
for example in test_examples:
    predicted_doc = nlp(example.text)
    for ent in example.reference.ents:
        y_true.append(ent.label_)
        # Dự đoán nhãn của mô hình
        pred_label = [e.label_ for e in predicted_doc.ents if e.text == ent.text]
        y_pred.append(pred_label[0] if pred_label else 'O')

# In ra các chỉ số Precision, Recall, F1-Score, và xử lý trường hợp không có dự đoán
print(classification_report(y_true, y_pred, zero_division=1))


              precision    recall  f1-score   support

       B-LOC       0.95      0.94      0.94      2274
      B-MISC       1.00      0.83      0.91        42
       B-ORG       0.99      0.92      0.95       859
       B-PER       0.96      0.92      0.94      1046
       I-LOC       0.95      0.92      0.93      1087
      I-MISC       0.83      0.73      0.78        41
       I-ORG       0.95      0.90      0.92      1358
       I-PER       1.00      0.60      0.75        15
           O       0.00      1.00      0.00         0

    accuracy                           0.92      6722
   macro avg       0.85      0.86      0.79      6722
weighted avg       0.96      0.92      0.94      6722



In [ ]:
# Lưu mô hình đã huấn luyện
nlp.to_disk("IE Model")
print("Mô hình đã được lưu thành công!")


Mô hình đã được lưu thành công!


## Test model (Run here if having model)

In [1]:
import spacy

# Giả sử mô hình đã được huấn luyện và lưu trữ tại path_to_save_model
nlp = spacy.load("IE Model")

# Hàm format kết quả thành định dạng <label> entity
def format_entities(doc):
    if not doc.ents:
        return "No entities found"  # Nếu không có thực thể nào, trả về thông báo
    formatted_output = []
    for ent in doc.ents:
        formatted_output.append(f"<{ent.label_}> {ent.text}")
    return " ".join(formatted_output)

# Danh sách các câu cần kiểm tra
test_sentences = [
    "Ông Nguyễn Văn An là một giáo viên tại trường Đại học Bách Khoa.",
    "Hà Nội là thủ đô của Việt Nam.",
    "Công ty FPT Technology là một trong những công ty công nghệ lớn nhất ở Việt Nam.",
    "Sự kiện Techfest 2024 sẽ diễn ra tại Hà Nội.",
    "Facebook đã mở văn phòng mới tại Singapore.",
    "Bà Nguyễn Thị Mai Châu là giám đốc của ngân hàng Techcombank.",
    "Thái Lan và Việt Nam sẽ thi đấu tại sân vận động Mỹ Đình.",
    "Ông Nguyễn Văn Bé vừa được trao giải thưởng sáng tạo của năm.",
    "Trời hôm nay đẹp và có nắng.",
    "Vào năm 2025, Google sẽ mở thêm một văn phòng tại thành phố Hồ Chí Minh và có kế hoạch hợp tác với Đại học Quốc gia Hà Nội."
]

# Xử lý từng câu trong danh sách test_sentences
for test_text in test_sentences:
    doc = nlp(test_text)
    formatted_output = format_entities(doc)
    print(f"Input: {test_text}\nOutput: {formatted_output}\n")


C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'vi_pipeline' (0.0.0) was trained with spaCy v3.7.5 and may not be 100% compatible with the current version (3.8.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Input: Ông Nguyễn Văn An là một giáo viên tại trường Đại học Bách Khoa.
Output: <B-PER> Nguyễn Văn An <B-ORG> trường <I-ORG> Đại học <I-ORG> Bách Khoa

Input: Hà Nội là thủ đô của Việt Nam.
Output: <B-LOC> Hà Nội <B-LOC> Việt Nam

Input: Công ty FPT Technology là một trong những công ty công nghệ lớn nhất ở Việt Nam.
Output: <B-ORG> Công ty <I-ORG> FPT <B-LOC> Việt Nam

Input: Sự kiện Techfest 2024 sẽ diễn ra tại Hà Nội.
Output: <B-LOC> Hà Nội

Input: Facebook đã mở văn phòng mới tại Singapore.
Output: <B-LOC> Singapore

Input: Bà Nguyễn Thị Mai Châu là giám đốc của ngân hàng Techcombank.
Output: <B-PER> Nguyễn Thị Mai Châu

Input: Thái Lan và Việt Nam sẽ thi đấu tại sân vận động Mỹ Đình.
Output: <B-LOC> Thái Lan <B-LOC> Việt Nam

Input: Ông Nguyễn Văn Bé vừa được trao giải thưởng sáng tạo của năm.
Output: <B-PER> Nguyễn Văn Bé

Input: Trời hôm nay đẹp và có nắng.
Output: No entities found

Input: Vào năm 2025, Google sẽ mở thêm một văn phòng tại thành phố Hồ Chí Minh và có kế hoạch hợ

In [ ]:
# test_sentences = [
#     "Nguyễn Văn A là một giáo viên tại trường Đại học Bách Khoa.",
#     "Hà Nội là thủ đô của Việt Nam.",
#     "Công ty FPT là một trong những công ty công nghệ lớn nhất ở Việt Nam.",
#     "Sự kiện Techfest 2024 sẽ diễn ra tại Hà Nội.",
#     "Facebook đã mở văn phòng mới tại Singapore.",
#     "Bà Nguyễn Thị Mai Châu là giám đốc của Techcombank.",
#     "Thái Lan và Việt Nam sẽ thi đấu tại sân vận động Mỹ Đình.",
#     "Nguyễn Văn B vừa được trao giải thưởng sáng tạo của năm.",
#     "Trời hôm nay đẹp và có nắng.",
#     "Vào năm 2025, Google sẽ mở thêm một văn phòng tại thành phố Hồ Chí Minh và có kế hoạch hợp tác với Đại học Quốc gia Hà Nội."
# ]

# Using pre-trained model

In [1]:
!pip install transformers datasets torch scikit-learn


  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
Using cached transformers-4.46.3-py3-none-any.whl (10.0 MB)
Using cached datasets-3.1.0-py3-none-any.whl (480 kB)


## Read dataset

In [2]:
from datasets import Dataset
from sklearn.model_selection import train_test_split

def read_conll(file_path):
    tokens, ner_tags = [], []
    sentences = []

    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()
            if not line:  # End line
                if tokens:
                    sentences.append({"tokens": tokens, "ner_tags": ner_tags})
                    tokens, ner_tags = [], []
            else:
                parts = line.split("\t")
                if len(parts) != 2:  # check format
                    print(f"Skipping invalid line: {line}")
                    continue
                word, label = parts
                tokens.append(word)
                ner_tags.append(label)
        if tokens:
            sentences.append({"tokens": tokens, "ner_tags": ner_tags})

    return sentences


file_path = "training_data.conll"
data = read_conll(file_path)

# split train/test (80% train, 20% test)
train_sentences, test_sentences = train_test_split(data, test_size=0.2, random_state=42)

# Change data Hugging Face Dataset
train_data = Dataset.from_list(train_sentences)
test_data = Dataset.from_list(test_sentences)

print(f"Number of training sentences: {len(train_data)}")
print(f"Number of testing sentences: {len(test_data)}")


C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of training sentences: 5949
Number of testing sentences: 1488


### Check label

In [6]:
label_list = ['O', 'B-MISC', 'I-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG']
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

def encode_labels(examples):
    # Chuyển đổi nhãn và đảm bảo không có nhãn lạ
    examples["labels"] = [
        [label_to_id.get(label, label_to_id['O']) for label in tags]  # Gán giá trị mặc định 'O' nếu nhãn không có trong label_to_id
        for tags in examples["ner_tags"]
    ]
    return examples

train_data = train_data.map(encode_labels)
test_data = test_data.map(encode_labels)



Map: 100%|██████████| 1488/1488 [00:00<00:00, 6602.63 examples/s]


In [7]:
# Kiểm tra các nhãn trong dữ liệu
all_labels = set()
for example in train_data:
    all_labels.update(example["ner_tags"])

print("Labels found in train data:", all_labels)


Labels found in train data: {'I-ORG', 'I-MISC', 'B-ORG', 'I-LOC', 'O', 'B-PER', 'I-PER', 'B-MISC', 'B-LOC'}


## Train with pre-trained model

In [38]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [3]:
!pip install --upgrade transformers
!pip install --upgrade torch


### Process data

In [8]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
#mô hình pretrained
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))

# Tokenize
def tokenize_and_align_labels(examples):
    # Tokenize words
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding=True,
        is_split_into_words=True,
        max_length=128
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # ID of words

        # Label it with the corresponding word and use -100 for padding or special tokens
        label_ids = []
        for word_id in word_ids:
            if word_id is None:  #not word
                label_ids.append(-100)
            else:
                label_ids.append(label_to_id[label[word_id]])  # label for word
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply tokenization to the training and test sets
train_data = train_data.map(tokenize_and_align_labels, batched=True)
test_data = test_data.map(tokenize_and_align_labels, batched=True)

# Kiểm tra một ví dụ dữ liệu
print(train_data[0])


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 1488/1488 [00:00<00:00, 3202.02 examples/s]

{'tokens': ['Tàu thuyền', 'hoạt động', 'trong', 'các', 'vùng', 'nguy hiểm', 'này', 'có', 'khả năng', 'chịu', 'tác động', 'của', 'giông', ',', 'lốc', ',', 'gió', 'mạnh', ',', 'sóng', 'lớn.Từ', 'đầu', 'năm', 'đến', 'nay', ',', 'Biển Đông', 'xuất hiện', '8', 'cơn', 'bão', '.'], 'ner_tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O'], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -

### Function to Evaluate model

In [7]:
from sklearn.metrics import classification_report, accuracy_score

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=2)  # Lấy nhãn dự đoán từ logits

    # Chuyển đổi nhãn từ ID sang tên nhãn
    true_labels = [[id_to_label[l] for l in label if l != -100] for label in labels]
    pred_labels = [[id_to_label[p] for (p, l) in zip(pred, label) if l != -100] for pred, label in zip(predictions, labels)]

    # Flatten danh sách các nhãn
    flat_true_labels = [label for sublist in true_labels for label in sublist]
    flat_pred_labels = [label for sublist in pred_labels for label in sublist]

    # Tính toán độ chính xác và các chỉ số khác
    report = classification_report(flat_true_labels, flat_pred_labels, output_dict=True, zero_division=1)
    accuracy = accuracy_score(flat_true_labels, flat_pred_labels)

    return {
        "accuracy": accuracy,
        "precision": report["macro avg"]["precision"],
        "recall": report["macro avg"]["recall"],
        "f1": report["macro avg"]["f1-score"],
    }

## Train model

### **The model has been trained**

In [44]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to=None,
    no_cuda=False,
    fp16=True,
    logging_dir="./logs",
    logging_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics,
)


trainer.train()

# Đánh giá mô hình trên tập kiểm tra
# eval_results = trainer.evaluate()



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.142400,0.095661,0.971374,0.724338,0.675784,0.679329
2,0.089900,0.082875,0.976432,0.780214,0.706150,0.729244
3,0.050300,0.084963,0.975760,0.867934,0.731908,0.748887


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

TrainOutput(global_step=2232, training_loss=0.08555971764321822, metrics={'train_runtime': 435.7074, 'train_samples_per_second': 40.961, 'train_steps_per_second': 5.123, 'total_flos': 1165914586434816.0, 'train_loss': 0.08555971764321822, 'epoch': 3.0})

### **Corrected a few warnings to complete the model**

In [48]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to=None,
    no_cuda=False,
    fp16=True,
    logging_dir="./logs",
    logging_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics,
)


trainer.train()

# Đánh giá mô hình trên tập kiểm tra
# eval_results = trainer.evaluate()



Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.042700,0.098655,0.976208,0.818290,0.778955,0.789776
2,0.035900,0.104832,0.975666,0.821237,0.782935,0.796586
3,0.016700,0.114980,0.976350,0.837258,0.804793,0.813246


TrainOutput(global_step=2232, training_loss=0.02988650935525108, metrics={'train_runtime': 428.9856, 'train_samples_per_second': 41.603, 'train_steps_per_second': 5.203, 'total_flos': 1165914586434816.0, 'train_loss': 0.02988650935525108, 'epoch': 3.0})

In [51]:
# Save model
model.save_pretrained("./saved_model_transformer_T2")
tokenizer.save_pretrained("./saved_model_transformer_T2")


('./saved_model_transformer_T2/tokenizer_config.json',
 './saved_model_transformer_T2/special_tokens_map.json',
 './saved_model_transformer_T2/vocab.txt',
 './saved_model_transformer_T2/added_tokens.json',
 './saved_model_transformer_T2/tokenizer.json')

## **Reuse the trained model to test the data (Run here if having model)**


In [13]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

# Load the trained model and tokenizer
model = AutoModelForTokenClassification.from_pretrained("./saved_model_transformer_T2")
tokenizer = AutoTokenizer.from_pretrained("./saved_model_transformer_T2")

# Đảm bảo rằng mô hình và dữ liệu được tải lên GPU (nếu có)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Hàm extract_information sử dụng mô hình đã huấn luyện
def extract_information(sentence):
    # Tokenize sentence
    tokenized_inputs = tokenizer(
        [sentence],  # Input là một câu
        is_split_into_words=False,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    )

    # Tensor for device
    tokenized_inputs = {key: value.to(device) for key, value in tokenized_inputs.items()}

    # predict label
    with torch.no_grad():
        outputs = model(**tokenized_inputs)

    predictions = outputs.logits.argmax(axis=2)

    # ID to label
    tokens = tokenizer.convert_ids_to_tokens(tokenized_inputs['input_ids'][0])
    pred_labels = [id_to_label[p.item()] for p in predictions[0]]

    # Filter data
    extracted_info = []
    for token, label in zip(tokens, pred_labels):
        if label != "O":  # Bỏ qua các token không có nhãn
            extracted_info.append((token, label))

    result = {}
    for token, label in extracted_info:
        entity_type = label.split("-")[-1]  # Bỏ tiền tố B- hoặc I-
        if entity_type not in result:
            result[entity_type] = []
        result[entity_type].append(token)

    return result

# Hàm xử lý các câu để in ra kết quả
def process_sentences(sentences):
    for idx, sentence in enumerate(sentences, 1):
        print(f"Sentence {idx}: {sentence}")
        result = extract_information(sentence)
        output = []
        for entity_type, entities in result.items():
            output.append(f"<{entity_type}> {' '.join(entities)}")
        print(" ".join(output))
        print("-" * 50)



In [14]:
test_sentences = [
    "Ông Nguyễn Văn An là một giáo viên tại trường Đại học Bách Khoa Thành phố Hồ Chí Minh.",
    "Hà Nội là thủ đô của Việt Nam.",
    "Công ty FPT Technology là một trong những công ty công nghệ lớn nhất ở Việt Nam.",
    "Sự kiện Techfest 2024 sẽ diễn ra tại Hà Nội.",
    "Facebook đã mở văn phòng mới tại Singapore.",
    "Bà Nguyễn Thị Mai Châu là giám đốc của ngân hàng Techcombank.",
    "Thái Lan và Việt Nam sẽ thi đấu tại sân vận động Mỹ Đình.",
    "Ông Nguyễn Văn Bé vừa được trao giải thưởng sáng tạo của năm.",
    "Trời hôm nay đẹp và có nắng.",
    "Vào năm 2025, Google sẽ mở thêm một văn phòng tại thành phố Hồ Chí Minh và có kế hoạch hợp tác với Đại học Quốc gia Hà Nội."
]

process_sentences(test_sentences)

Sentence 1: Ông Nguyễn Văn An là một giáo viên tại trường Đại học Bách Khoa Thành phố Hồ Chí Minh.
<PER> Nguyễn Văn An <ORG> trường Đại học B ##ách Khoa Thành phố Hồ Chí Minh
--------------------------------------------------
Sentence 2: Hà Nội là thủ đô của Việt Nam.
<LOC> Hà Nội Việt Nam
--------------------------------------------------
Sentence 3: Công ty FPT Technology là một trong những công ty công nghệ lớn nhất ở Việt Nam.
<ORG> Công ty F ##P ##T Technology <LOC> Việt Nam
--------------------------------------------------
Sentence 4: Sự kiện Techfest 2024 sẽ diễn ra tại Hà Nội.
<LOC> Hà Nội
--------------------------------------------------
Sentence 5: Facebook đã mở văn phòng mới tại Singapore.
<LOC> Singapore
--------------------------------------------------
Sentence 6: Bà Nguyễn Thị Mai Châu là giám đốc của ngân hàng Techcombank.
<PER> Nguyễn Thị Mai Châu <ORG> Tech ##com ##bank
--------------------------------------------------
Sentence 7: Thái Lan và Việt Nam sẽ thi đấu t